In [5]:
from skimage import io, color, img_as_ubyte, util, segmentation, filters
import cv2
import numpy as np
import warnings
import scipy
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from scipy.spatial import distance
import math

def showimg(img, title="Obraz", cmap="gray"):
    plt.figure(figsize=(15,15))
    plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    plt.show()
tab2 = []
coin_count = 0

def del_empty_obj(tab_obj,last_obj):
    x=0
    global coin_count
    while x <= last_obj:
        tab2.append(x)
        x+=1
    for i in range(tab_obj.shape[0]):
        for j in range(tab_obj.shape[1]):
            if(tab_obj[i][j] != 0):
                tab2[tab_obj[i][j]] += 1     
    for y in range(len(tab2)):
        if(tab2[y]<50):
            tab2[y] = 0
        if(tab2[y]!=0):
            coin_count += 1
    
def region_growing(im,tab_obj):
    obj=1
    last_obj=obj
    for i in range(im.shape[0]):
        for j in range(im.shape[1]):
            if(im[i][j]!=0):
                if(im[i][j]==im[i][j-1] or im[i][j]==im[i-1][j] or im[i][j]==im[i][j+1] or im[i][j]==im[i+1][j] or im[i][j]==im[i-1][j-1] or im[i][j]==im[i-1][j+1] or im[i][j]==im[i+1][j-1] or im[i][j]==im[i+1][j+1]):
                    
                    #sprawdzanie czy pixele obok są w jakimś obiekcie
                    if(im[i][j]==im[i][j-1]):
                        obj = tab_obj[i][j-1]    
                    if(im[i][j]==im[i-1][j]):
                        obj = tab_obj[i-1][j] 
                    if(im[i][j]==im[i-1][j-1]):
                        obj = tab_obj[i][j-1]    
                    if(im[i][j]==im[i-1][j+1] and tab_obj[i-1][j+1]>0):
                        obj = tab_obj[i-1][j+1]    
         
                    if(obj>last_obj):
                        last_obj = obj  
                    
                    if(tab_obj[i][j] == 0 and im[i][j] != 0):
                        a = i
                        b = j
                        while im[a][b] != 0:    
                            if(im[a][b]==im[a-1][b]):
                                obj = tab_obj[a-1][b]
                                break
                            if(im[a][b]==im[a-1][b+1] and tab_obj[a-1][b+1]>0):
                                obj = tab_obj[a-1][b+1]
                                break
                            b=b+1
                        
                    
                    tab_obj[i][j] = obj
                    

                    if(im[i][j+1]==0):
                        obj = last_obj+1
    
    del_empty_obj(tab_obj,last_obj)
    

           
warnings.filterwarnings('ignore')    
    
url ='zdjecia/8.jpg'
im = io.imread(url)
im = cv2.resize(im,(362,512))

#showimg(im,title="Obraz oryginalny")



im2 = color.rgb2gray(im)
im2 = img_as_ubyte(im2)
im2 = np.invert(im2)
im2 = cv2.blur(im2, (5,5))
#showimg(im2,title="Obraz po rozmyciu")
tab_obj = np.zeros_like(im2)

th = 150
th,bim = cv2.threshold(im2, thresh=th, maxval=255, type=cv2.THRESH_OTSU)

#showimg(bim,title="Obraz po binaryzacji")

element = np.ones((3,3),np.uint8)
#wynzacznie tła
bg = cv2.dilate(bim,kernel=element,iterations=3)

dt = cv2.distanceTransform(bim, distanceType=cv2.DIST_L2, maskSize=3)
ret, fg = cv2.threshold(dt,0.5*dt.max(),255,0)

#showimg(dt,"Odległość od środka")

#wyznaczanie części będącej na pewno monetą
fg = np.uint8(fg)


#showimg(fg,"rozdzielone monety")

#wyrównanie obiektów
seg = cv2.dilate(fg,kernel=element,iterations=3)
seg = cv2.morphologyEx(seg, op=cv2.MORPH_OPEN, kernel=element, iterations=5)

#segemntacja
region_growing(seg,tab_obj)

print(f"Wykryto {coin_count} monet")


#zliczanie pola monet

def coin_area(url):
    im = io.imread(url)
    im = cv2.resize(im,(362,512))
    im = color.rgb2gray(im)
    im = img_as_ubyte(im)
    im = np.invert(im)
    im = cv2.blur(im, (5,5))    
    th = 150
    th,bim = cv2.threshold(im, thresh=th, maxval=255, type=cv2.THRESH_OTSU)

    dt = cv2.distanceTransform(bim, distanceType=cv2.DIST_L2, maskSize=3)
    ret, fg = cv2.threshold(dt,0.5*dt.max(),255,0)
    fg = np.uint8(fg)
    size = 0

    for i in range(bim.shape[0]):
            for j in range(bim.shape[1]):
                if bim[i][j] != 0:
                    size+=1

    return size


url = 'zdjecia/2zl.jpg'
dwa_zl = coin_area(url)

url = 'zdjecia/1zl.jpg'
jeden_zl = coin_area(url)

url = 'zdjecia/50gr.jpg'
piedziesiat_gr = coin_area(url)

url = 'zdjecia/20gr.jpg'
dwadziescia_gr = coin_area(url)

url = 'zdjecia/10gr.jpg'
dziesiec_gr = coin_area(url)

url = 'zdjecia/5gr.jpg'
piec_gr = coin_area(url)

url = 'zdjecia/2gr.jpg'
dwa_gr = coin_area(url)

url = 'zdjecia/1gr.jpg'
jeden_gr = coin_area(url)                

obj_nr, labels = ndi.label(fg)

sz = np.bincount(obj_nr.ravel())

value = 0
i=1
total = [0,0,0,0,0,0,0,0,0]

while i < len(sz):
    if sz[i]>=385:
        value+=500
        total[0] += 1
#         print("5 zl")
    elif sz[i]>=330 and sz[i]<385:
        value+=100
        total[2] += 1
#         print("1 zl")
    elif sz[i]>=280 and sz[i]<330:
        value+=200
        total[1] += 1
#         print("2 zl")
    elif sz[i]>=215 and sz[i]<280:
        value+=50
        total[3] += 1
#         print("50 gr")
    elif sz[i]>=170 and sz[i]<215:
        value+=5
        total[6] += 1
#         print("5 gr")
    elif sz[i]>=125 and sz[i]<170:
        value+=20
        total[4] += 1
#         print("20 gr")
    elif sz[i]>=110 and sz[i]<125:
        value+=2
        total[7] += 1
#         print("2 gr")
    elif sz[i]>70 and sz[i]<110:
        value+=10
        total[5] += 1
#         print("10 gr")
    elif sz[i]<=70 and sz[i]>20:
        value+=1
        total[8] += 1
#         print("1 gr")
#    else:
#         print("Skip")
    i+=1


def area_percent(tab):
    i = 1
    T = []
    while i < len(tab):
        if tab[i] != 0:
            T.append(((tab[i]*3)/fg.size)*100)
        i += 1
    return T    

def center(im,obj_nr,obj,tab):
    x = 0
    y = 0
    licznik = 0
    for i in range(im.shape[0]):
        for j in range(im.shape[1]):
            if obj_nr[i][j] == obj:
                x += i
                y += j
                licznik +=1
    x = x/licznik
    y = y/licznik
    tab.append(x)
    tab.append(y)
    return tab


def Feret(im,obj_nr,obj):
    max_x = 0
    min_x = 362
    x = 0
    max_y = 0
    min_y = 512
    y = 0
    for i in range(im.shape[0]):
        for j in range(im.shape[1]):
            if obj_nr[i][j] == obj:
                if i < min_y:
                    min_y = i
                if i > max_y:
                    max_y = i
                if j < min_x:
                    min_x = j
                if j > max_x:
                    max_x = j
    x = max_x - min_x
    y = max_y - min_y
    
    return x/y
    
def getFigure(obj_nr, obj):
    
    points = []
    for y in range(obj_nr.shape[0]):
        for x in range(obj_nr.shape[1]):
            if obj_nr[y,x] == obj:
                points.append((y,x))

    return points

def BlairBliss(points,tab):
    
    s = len(points)
    mx = tab[1]
    my = tab[0]
    
    r = 0
    for point in points:
         r = r + distance.euclidean(point,(my,mx))**2
            
    return s/(math.sqrt(2*math.pi*r))

# print(dwa_zl)
# print(jeden_zl)
# print(piedziesiat_gr)
# print(dwadziescia_gr)
# print(dziesiec_gr)
# print(piec_gr)
# print(dwa_gr)
# print(jeden_gr)

print()
print("Ilosc monet ulozona wartosciami od 5zl w dol: ")
print(total)
print()
print(f"Suma : {value/100} zl")

T = area_percent(tab2)
i = 0
print()
p_all = 0
while i < len(T):
    pts = getFigure(obj_nr, i+1)
    tab = []
    print("Obiekt "+str(i+1)+":")
    print("   Zajmuje: "+str(round(T[i],2))+"% zdjecia")
    center(fg,obj_nr,i+1,tab)
    print("   Srodek geometryczny: ("+str(int(tab[0]))+","+str(int(tab[1]))+")")
    feret = Feret(fg,obj_nr,i+1)
    print("   Feret: "+str(feret))
    blairbliss = BlairBliss(pts,tab)
    print("   Blair-Bliss: "+str(blairbliss))
    p_all += T[i]
    i += 1
print()
print("Obiekty zajmuja lacznie okolo "+str(round(p_all,2))+"% miejsca na zdjeciu")
print()

#print(sz)

Wykryto 12 monet

Ilosc monet ulozona wartosciami od 5zl w dol: 
[1, 1, 1, 2, 1, 1, 2, 1, 2]

Suma : 9.44 zl

Obiekt 1:
   Zajmuje: 0.31% zdjecia
   Srodek geometryczny: (165,256)
   Feret: 1.0
   Blair-Bliss: 0.9994303613566252
Obiekt 2:
   Zajmuje: 0.65% zdjecia
   Srodek geometryczny: (205,167)
   Feret: 0.9333333333333333
   Blair-Bliss: 0.9992299419219861
Obiekt 3:
   Zajmuje: 0.47% zdjecia
   Srodek geometryczny: (221,134)
   Feret: 0.8461538461538461
   Blair-Bliss: 0.996174647634484
Obiekt 4:
   Zajmuje: 0.76% zdjecia
   Srodek geometryczny: (229,203)
   Feret: 0.9444444444444444
   Blair-Bliss: 0.9912595272346052
Obiekt 5:
   Zajmuje: 0.32% zdjecia
   Srodek geometryczny: (236,170)
   Feret: 0.9
   Blair-Bliss: 0.982640334525419
Obiekt 6:
   Zajmuje: 0.79% zdjecia
   Srodek geometryczny: (253,144)
   Feret: 0.8888888888888888
   Blair-Bliss: 0.9955025446052397
Obiekt 7:
   Zajmuje: 1.27% zdjecia
   Srodek geometryczny: (264,189)
   Feret: 0.8461538461538461
   Blair-Bliss: 0.9